# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
import pandas as pd

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

In [2]:
# auth = InteractiveLoginAuthentication()
ws = Workspace.from_config()
# 
# choose a name for experiment
experiment_name = 'classification-hyperdrive'
project_folder = './capstoneProject'
experiment=Experiment(ws, experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()


Workspace name: quick-starts-ws-162678
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-162678


In [ ]:
# import os

# subscription_id = os.getenv("SUBSCRIPTION_ID", default="9b72f9e6-56c5-4c16-991b-19c652994860")
# resource_group = os.getenv("RESOURCE_GROUP", default="aml-quickstarts-162678")
# workspace_name = os.getenv("WORKSPACE_NAME", default="quick-starts-ws-162678")
# workspace_region = os.getenv("WORKSPACE_REGION", default="southcentralus")


# from azureml.core import Workspace

# try:
#     ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#     # write the details of the workspace to a configuration file to the notebook library
#     ws.write_config()
#     print("Workspace configuration succeeded. Skip the workspace creation steps below")
# except:
#     print("Workspace not accessible. Change your parameters or create a new workspace below")

In [ ]:
# !az login

In [ ]:
# # ws = Workspace.from_config()

# # choose a name for experiment
# experiment_name = 'classification-automl'
# project_folder = './capstoneProject'
# experiment=Experiment(ws, experiment_name)


# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep = '\n')

# run = experiment.start_logging()

# Create or Attach an AmlCompute cluster

In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyperdiveCompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D16_v3',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'dedicated', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": uniform(0.03, 1),
    "--max_iter": choice(50, 100, 150, 200, 300)
})

if "training" not in os.listdir():
    os.mkdir("./training")
    
#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory=".", 
              compute_target=amlcompute_cluster_name, 
              entry_script="train.py",
             pip_packages=['pandas'])

hyperdrive_run_config = HyperDriveConfig(estimator=est, 
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy, 
                                         primary_metric_name='Accuracy', 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=20, 
                                         max_concurrent_runs=4)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get the best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
#TODO: Save the best model

## Register best model
metrics = best_run.get_metrics()
print(metrics)
model = best_run.register_model(model_name = 'hyperdrive_model', model_path='./outputs/hyperdrive_model.pkl')
model